Meta-Learning for Few-Shot Log Anomaly Detection in Extreme Imbalance Settings

In [43]:
import os
import sys
import pickle
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.optim import Adam, SGD
import torch.multiprocessing as mp

# Import GradScaler with backward compatibility
try:
    from torch.amp import autocast, GradScaler
    USE_NEW_AMP_API = True
except ImportError:
    from torch.cuda.amp import autocast, GradScaler
    USE_NEW_AMP_API = False

from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [44]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Enable optimizations
torch.backends.cudnn.benchmark = True
if hasattr(torch, 'set_float32_matmul_precision'):
    torch.set_float32_matmul_precision('medium')

ROOT = Path(r"C:\Computer Science\AIMLDL\log-anomaly-detection")
FEAT_PATH = ROOT / "features"
RESULTS_PATH = ROOT / "results" / "meta_learning"
MODELS_PATH = ROOT / "models" / "meta_learning"

RESULTS_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)

Using device: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.59 GB


In [45]:
feat_file = FEAT_PATH / "enhanced_imbalanced_features.pkl"
with open(feat_file, 'rb') as f:
    feat_data = pickle.load(f)
    data_dict = feat_data['hybrid_features_data']
    num_classes = feat_data['config'].get('num_classes', 2)

split_file = FEAT_PATH / "enhanced_cross_source_splits.pkl"
with open(split_file, 'rb') as f:
    split_data = pickle.load(f)
    splits = split_data['splits']

print(f"Loaded {len(data_dict)} sources")
print(f"Classes: {num_classes}")

LABEL_MAP = {0: 'normal', 1: 'anomaly'}

Loaded 16 sources
Classes: 2


In [46]:
def calculate_metrics(y_true, y_pred, y_proba=None):
    metrics = {}
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['balanced_acc'] = balanced_accuracy_score(y_true, y_pred)
    metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['mcc'] = matthews_corrcoef(y_true, y_pred)
    
    per_class = {}
    for cls in np.unique(np.concatenate([y_true, y_pred])):
        y_true_bin = (y_true == cls).astype(int)
        y_pred_bin = (y_pred == cls).astype(int)
        if y_true_bin.sum() > 0:
            per_class[int(cls)] = {
                'precision': precision_score(y_true_bin, y_pred_bin, zero_division=0),
                'recall': recall_score(y_true_bin, y_pred_bin, zero_division=0),
                'f1': f1_score(y_true_bin, y_pred_bin, zero_division=0),
                'support': int(y_true_bin.sum())
            }
    metrics['per_class'] = per_class
    
    if y_proba is not None and len(np.unique(y_true)) == 2:
        try:
            metrics['auroc'] = roc_auc_score(y_true, y_proba[:, 1])
            metrics['auprc'] = average_precision_score(y_true, y_proba[:, 1])
        except:
            metrics['auroc'] = 0.0
            metrics['auprc'] = 0.0
    else:
        metrics['auroc'] = 0.0
        metrics['auprc'] = 0.0
    
    return metrics

In [47]:
def create_few_shot_episode(X, y, n_way, k_shot, q_query, balance=True):
    classes = np.unique(y)
    if len(classes) < n_way:
        return None, None, None, None
    
    selected_classes = np.random.choice(classes, n_way, replace=False)
    
    support_X, support_y = [], []
    query_X, query_y = [], []
    
    for cls in selected_classes:
        cls_indices = np.where(y == cls)[0]
        
        if len(cls_indices) < k_shot + q_query:
            if balance:
                return None, None, None, None
            else:
                available = len(cls_indices)
                k_use = min(k_shot, available // 2)
                q_use = available - k_use
                if k_use == 0 or q_use == 0:
                    return None, None, None, None
        else:
            k_use = k_shot
            q_use = q_query
        
        selected = np.random.choice(cls_indices, k_use + q_use, replace=False)
        support_indices = selected[:k_use]
        query_indices = selected[k_use:k_use + q_use]
        
        support_X.append(X[support_indices])
        support_y.append(y[support_indices])
        query_X.append(X[query_indices])
        query_y.append(y[query_indices])
    
    support_X = np.vstack(support_X)
    support_y = np.concatenate(support_y)
    query_X = np.vstack(query_X)
    query_y = np.concatenate(query_y)
    
    shuffle_support = np.random.permutation(len(support_y))
    support_X = support_X[shuffle_support]
    support_y = support_y[shuffle_support]
    
    shuffle_query = np.random.permutation(len(query_y))
    query_X = query_X[shuffle_query]
    query_y = query_y[shuffle_query]
    
    return support_X, support_y, query_X, query_y

In [48]:
def create_balanced_imbalanced_episode(X, y, k_shot_range=(3, 10), q_query=15):
    """Create episodes with controlled imbalance and stratification"""
    classes = np.unique(y)
    if len(classes) != 2:
        return None, None, None, None
    
    class_counts = [np.sum(y == cls) for cls in classes]
    minority_cls = classes[np.argmin(class_counts)]
    majority_cls = classes[np.argmax(class_counts)]
    
    minority_indices = np.where(y == minority_cls)[0]
    majority_indices = np.where(y == majority_cls)[0]
    
    k_minority = min(k_shot_range[1], len(minority_indices) // 3)
    k_majority = min(k_minority * 3, len(majority_indices) // 3)

    if len(minority_indices) < k_minority + q_query:
        # Adjust k_minority and q_query to fit available samples
        total_minority = len(minority_indices)
        if total_minority < k_shot_range[0] + 1:  # Need at least k_shot_range[0] + 1 sample
            return None, None, None, None
        k_minority = max(k_shot_range[0], total_minority // 2)
        q_query_minority = total_minority - k_minority
        if q_query_minority < 1:
            return None, None, None, None
    else:
        q_query_minority = q_query
    
    if len(majority_indices) < k_majority + q_query:
        # Adjust k_majority and q_query to fit available samples
        total_majority = len(majority_indices)
        if total_majority < k_shot_range[0] + 1:
            return None, None, None, None
        k_majority = max(k_shot_range[0], total_majority // 2)
        q_query_majority = total_majority - k_majority
        if q_query_majority < 1:
            return None, None, None, None
    else:
        q_query_majority = q_query
    
    if k_minority < k_shot_range[0] or k_majority < k_shot_range[0]:
        return None, None, None, None
    
    minority_selected = np.random.choice(minority_indices, k_minority + q_query_minority, replace=False)
    majority_selected = np.random.choice(majority_indices, k_majority + q_query_majority, replace=False)
    
    support_X = np.vstack([X[minority_selected[:k_minority]], X[majority_selected[:k_majority]]])
    support_y = np.concatenate([y[minority_selected[:k_minority]], y[majority_selected[:k_majority]]])
    
    query_X = np.vstack([X[minority_selected[k_minority:]], X[majority_selected[k_majority:]]])
    query_y = np.concatenate([y[minority_selected[k_minority:]], y[majority_selected[k_majority:]]])
    
    shuffle_support = np.random.permutation(len(support_y))
    support_X = support_X[shuffle_support]
    support_y = support_y[shuffle_support]
    
    shuffle_query = np.random.permutation(len(query_y))
    query_X = query_X[shuffle_query]
    query_y = query_y[shuffle_query]
    
    return support_X, support_y, query_X, query_y

def create_imbalanced_episode(X, y, minority_k_shot, majority_k_shot, q_query_per_class):
    classes = np.unique(y)
    if len(classes) != 2:
        return None, None, None, None
    
    class_counts = [np.sum(y == cls) for cls in classes]
    minority_cls = classes[np.argmin(class_counts)]
    majority_cls = classes[np.argmax(class_counts)]
    
    minority_indices = np.where(y == minority_cls)[0]
    majority_indices = np.where(y == majority_cls)[0]
    
    if len(minority_indices) < minority_k_shot + q_query_per_class:
        return None, None, None, None
    if len(majority_indices) < majority_k_shot + q_query_per_class:
        return None, None, None, None
    
    minority_selected = np.random.choice(minority_indices, minority_k_shot + q_query_per_class, replace=False)
    majority_selected = np.random.choice(majority_indices, majority_k_shot + q_query_per_class, replace=False)
    
    support_X = np.vstack([X[minority_selected[:minority_k_shot]], X[majority_selected[:majority_k_shot]]])
    support_y = np.concatenate([y[minority_selected[:minority_k_shot]], y[majority_selected[:majority_k_shot]]])
    
    query_X = np.vstack([X[minority_selected[minority_k_shot:]], X[majority_selected[majority_k_shot:]]])
    query_y = np.concatenate([y[minority_selected[minority_k_shot:]], y[majority_selected[majority_k_shot:]]])
    
    shuffle_support = np.random.permutation(len(support_y))
    support_X = support_X[shuffle_support]
    support_y = support_y[shuffle_support]
    
    shuffle_query = np.random.permutation(len(query_y))
    query_X = query_X[shuffle_query]
    query_y = query_y[shuffle_query]
    
    return support_X, support_y, query_X, query_y

In [49]:
def prototypical_loss(embeddings, labels, n_way):
    classes = torch.unique(labels)
    prototypes = []
    
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = embeddings[cls_mask]
        prototype = cls_embeddings.mean(dim=0)
        prototypes.append(prototype)
    
    prototypes = torch.stack(prototypes)
    
    distances = torch.cdist(embeddings, prototypes, p=2)
    log_probs = F.log_softmax(-distances, dim=1)
    
    loss = F.nll_loss(log_probs, labels)
    return loss

def focal_loss(logits, labels, alpha=0.75, gamma=3.0):
    ce_loss = F.cross_entropy(logits, labels, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss
    return focal_loss.mean()

def contrastive_loss(embeddings, labels, temperature=0.5):
    embeddings = F.normalize(embeddings, dim=1)
    similarity_matrix = torch.matmul(embeddings, embeddings.T) / temperature
    
    labels = labels.contiguous().view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(device)
    
    logits_max, _ = torch.max(similarity_matrix, dim=1, keepdim=True)
    logits = similarity_matrix - logits_max.detach()
    
    exp_logits = torch.exp(logits)
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))
    
    mask_sum = mask.sum(1)
    mask_sum = torch.clamp(mask_sum, min=1.0)
    
    mean_log_prob_pos = (mask * log_prob).sum(1) / mask_sum
    loss = -mean_log_prob_pos.mean()
    
    return loss

def combined_meta_loss(embeddings, logits, labels, prototypes=None, 
                       alpha_focal=0.4, alpha_proto=0.3, alpha_contrastive=0.3):
    """Multi-task loss for better representation learning"""
    focal = focal_loss(logits, labels)
    
    if prototypes is not None:
        proto = prototypical_loss(embeddings, labels, len(prototypes))
    else:
        proto = torch.tensor(0.0).to(embeddings.device)
    
    contrastive = contrastive_loss(embeddings, labels, temperature=0.5)
    
    return alpha_focal * focal + alpha_proto * proto + alpha_contrastive * contrastive

In [50]:
class EpisodeCache:
    def __init__(self, cache_size=500):
        self.cache = {}
        self.cache_size = cache_size
        self.hits = 0
        self.misses = 0
    
    def get_key(self, source_name, k_min, k_maj, q_query, seed):
        return f"{source_name}_{k_min}_{k_maj}_{q_query}_{seed}"
    
    def get_episode(self, source_name, X, y, k_min, k_maj, q_query):
        cache_seed = np.random.randint(0, self.cache_size)
        key = self.get_key(source_name, k_min, k_maj, q_query, cache_seed)
        
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        
        self.misses += 1
        episode = create_balanced_imbalanced_episode(X, y, q_query=q_query)
        
        if episode[0] is not None and len(self.cache) < self.cache_size:
            self.cache[key] = episode
        
        return episode
    
    def get_stats(self):
        total = self.hits + self.misses
        hit_rate = self.hits / total if total > 0 else 0
        return f"Cache hit rate: {hit_rate:.2%} ({self.hits}/{total})"

In [51]:
def augment_support_set(support_X, support_y, augment_factor=2):
    """Augment minority class samples with noise and mixup"""
    minority_cls = np.argmin(np.bincount(support_y))
    minority_mask = support_y == minority_cls
    minority_X = support_X[minority_mask]
    
    augmented_X = []
    augmented_y = []
    
    for _ in range(augment_factor):
        noise = np.random.normal(0, 0.01, minority_X.shape)
        augmented_X.append(minority_X + noise)
        augmented_y.append(np.full(len(minority_X), minority_cls))
    
    for i in range(len(minority_X)):
        j = np.random.randint(len(minority_X))
        lam = np.random.beta(0.2, 0.2)
        mixed = lam * minority_X[i] + (1 - lam) * minority_X[j]
        augmented_X.append(mixed.reshape(1, -1))
        augmented_y.append([minority_cls])
    
    support_X_aug = np.vstack([support_X] + augmented_X)
    support_y_aug = np.concatenate([support_y] + augmented_y)
    
    return support_X_aug, support_y_aug

In [52]:
def adaptive_inner_steps(imbalance_ratio, base_steps=5):
    """Adjust inner loop steps based on task difficulty"""
    if imbalance_ratio > 100:
        return base_steps * 3
    elif imbalance_ratio > 20:
        return base_steps * 2
    return base_steps

class TaskAdaptiveLR:
    def __init__(self, base_lr=1e-2):
        self.base_lr = base_lr
    
    def get_lr(self, imbalance_ratio):
        """Adjust LR based on task characteristics"""
        if imbalance_ratio > 100:
            return self.base_lr * 0.5
        elif imbalance_ratio < 5:
            return self.base_lr * 1.5
        return self.base_lr

In [53]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.4):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim)
        self.bn1 = nn.BatchNorm1d(dim)
        self.fc2 = nn.Linear(dim, dim)
        self.bn2 = nn.BatchNorm1d(dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.fc1(x)))
        out = self.dropout(out)
        out = self.bn2(self.fc2(out))
        out += residual
        return F.relu(out)

class AttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, input_dim // 2),
            nn.Tanh(),
            nn.Linear(input_dim // 2, 1)
        )
    
    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=0)
        return (x * weights).sum(dim=0, keepdim=True)

def meta_network(input_dim, hidden_dims=[512, 256, 128], output_dim=128, dropout=0.4):
    layers = []
    prev_dim = input_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(prev_dim, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        prev_dim = hidden_dim
    layers.append(nn.Linear(prev_dim, output_dim))
    return nn.Sequential(*layers)

def classifier_head(input_dim, num_classes=2, dropout=0.4):
    return nn.Sequential(
        nn.Linear(input_dim, input_dim // 2),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(input_dim // 2, num_classes)
    )

def attention_pooling(input_dim):
    return nn.Sequential(
        nn.Linear(input_dim, 1),
        nn.Softmax(dim=1)
    )

In [54]:
def maml_inner_loop(model, support_X, support_y, inner_lr, inner_steps, loss_fn, use_amp=True):
    model_copy = ImprovedMetaLearner(model.input_dim, model.hidden_dims, model.embedding_dim, 
                                     model.dropout, model.num_classes).to(device)
    
    # Handle torch.compile state dict
    state_dict = model.state_dict()
    if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
        state_dict = {key.replace('_orig_mod.', ''): value for key, value in state_dict.items()}
    
    model_copy.load_state_dict(state_dict)
    
    optimizer = SGD(model_copy.parameters(), lr=inner_lr)
    
    support_X_tensor = torch.FloatTensor(support_X).to(device, non_blocking=True)
    support_y_tensor = torch.LongTensor(support_y).to(device, non_blocking=True)
    
    prev_loss = float('inf')
    tolerance = 1e-4
    
    for step in range(inner_steps):
        optimizer.zero_grad()
        
        if use_amp and torch.cuda.is_available():
            device_type = 'cuda' if USE_NEW_AMP_API else None
            with autocast(device_type=device_type) if USE_NEW_AMP_API else autocast():
                logits = model_copy.predict(support_X_tensor)
                loss = loss_fn(logits, support_y_tensor)
        else:
            logits = model_copy.predict(support_X_tensor)
            loss = loss_fn(logits, support_y_tensor)
        
        # Early stopping if converged
        if step > 2 and abs(prev_loss - loss.item()) < tolerance:
            break
        prev_loss = loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_copy.parameters(), 1.0)
        optimizer.step()
    
    return model_copy

def batched_maml_inner_loop(model, support_X, support_y, inner_lr, inner_steps, loss_fn, batch_size=32):
    """Batched inner loop for faster training"""
    model_copy = ImprovedMetaLearner(model.input_dim, model.hidden_dims, model.embedding_dim,
                                     model.dropout, model.num_classes).to(device)
    
    # Handle torch.compile state dict
    state_dict = model.state_dict()
    if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
        state_dict = {key.replace('_orig_mod.', ''): value for key, value in state_dict.items()}
    
    model_copy.load_state_dict(state_dict)
    
    optimizer = SGD(model_copy.parameters(), lr=inner_lr)
    
    support_X_tensor = torch.FloatTensor(support_X).to(device, non_blocking=True)
    support_y_tensor = torch.LongTensor(support_y).to(device, non_blocking=True)
    
    dataset_size = len(support_y)
    
    for step in range(inner_steps):
        indices = torch.randperm(dataset_size)
        
        for i in range(0, dataset_size, batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = support_X_tensor[batch_indices]
            y_batch = support_y_tensor[batch_indices]
            
            optimizer.zero_grad()
            
            device_type = 'cuda' if USE_NEW_AMP_API else None
            with autocast(device_type=device_type) if USE_NEW_AMP_API else autocast():
                logits = model_copy.predict(X_batch)
                loss = loss_fn(logits, y_batch)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_copy.parameters(), 1.0)
            optimizer.step()
    
    return model_copy

def memory_efficient_maml(model, support_X, support_y, inner_lr, inner_steps, loss_fn):
    params = {name: param.clone() for name, param in model.named_parameters()}
    
    support_X_tensor = torch.FloatTensor(support_X).to(device)
    support_y_tensor = torch.LongTensor(support_y).to(device)
    
    for step in range(inner_steps):
        embeddings = model(support_X_tensor)
        logits = model.classifier(embeddings)
        loss = loss_fn(logits, support_y_tensor)
        
        grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)
        
        for (name, param), grad in zip(model.named_parameters(), grads):
            param.data = param.data - inner_lr * grad
    
    result_params = {name: param.clone() for name, param in model.named_parameters()}
    
    for name, param in model.named_parameters():
        param.data = params[name].data
    
    return result_params

def process_single_task(encoder_state, classifier_state, source_name, source_features, source_labels, 
                       source_k_shots, inner_lr, inner_steps, q_query, device_id, input_dim, hidden_dims, 
                       embedding_dim, dropout, num_classes):
    local_device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")
    
    local_encoder = meta_network(input_dim, hidden_dims, embedding_dim, dropout).to(local_device)
    local_encoder.load_state_dict(encoder_state)
    local_encoder.input_dim = input_dim
    local_encoder.hidden_dims = hidden_dims
    local_encoder.output_dim = embedding_dim
    local_encoder.dropout = dropout
    
    local_classifier = classifier_head(embedding_dim, num_classes, dropout).to(local_device)
    local_classifier.load_state_dict(classifier_state)
    local_encoder.classifier = local_classifier
    
    X_source = source_features[source_name]
    y_source = source_labels[source_name]
    k_shots = source_k_shots[source_name]
    
    episode = create_imbalanced_episode(
        X_source, y_source, 
        k_shots['minority'], k_shots['majority'], q_query
    )
    
    if episode[0] is None:
        return None
    
    support_X, support_y, query_X, query_y = episode
    
    adapted_model = maml_inner_loop(
        local_encoder, support_X, support_y, 
        inner_lr, inner_steps, focal_loss
    )
    
    query_X_tensor = torch.FloatTensor(query_X).to(local_device)
    query_y_tensor = torch.LongTensor(query_y).to(local_device)
    
    query_embeddings = adapted_model(query_X_tensor)
    query_logits = adapted_model.classifier(query_embeddings)
    
    task_loss = focal_loss(query_logits, query_y_tensor)
    
    return task_loss.cpu()

In [55]:
def compute_prototypes(embeddings, labels):
    classes = torch.unique(labels)
    prototypes = []
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = embeddings[cls_mask]
        prototype = cls_embeddings.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes), classes

def refined_prototypes(embeddings, labels, momentum=0.9):
    """Exponential moving average for stable prototypes"""
    if not hasattr(refined_prototypes, 'proto_memory'):
        refined_prototypes.proto_memory = {}
    
    classes = torch.unique(labels)
    prototypes = []
    
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = embeddings[cls_mask]
        current_proto = cls_embeddings.mean(dim=0)
        
        cls_key = int(cls.item())
        if cls_key in refined_prototypes.proto_memory:
            refined_proto = (momentum * refined_prototypes.proto_memory[cls_key] + 
                           (1 - momentum) * current_proto)
        else:
            refined_proto = current_proto
        
        refined_prototypes.proto_memory[cls_key] = refined_proto.detach()
        prototypes.append(refined_proto)
    
    return torch.stack(prototypes)

In [56]:
OPTIMIZED_CONFIG = {
    'input_dim': 200,
    'hidden_dims': [512, 256, 128],
    'embedding_dim': 128,
    'dropout': 0.4,
    'meta_lr': 5e-4,
    'inner_lr': 5e-3,
    'inner_steps': 10,
    'meta_batch_size': 16,
    'focal_alpha': 0.75,
    'focal_gamma': 3.0,
    'augment_minority': True,
    'use_mixup': True,
    'use_amp': True,
    'use_cache': True,
    'use_batched_inner': False,
    'gradient_accumulation_steps': 1
}

# Speed-optimized config for 8GB GPU
SPEED_OPTIMIZED_CONFIG = {
    'input_dim': 200,
    'hidden_dims': [256, 128],
    'embedding_dim': 64,
    'dropout': 0.3,
    'meta_lr': 5e-4,
    'inner_lr': 5e-3,
    'inner_steps': 8,
    'meta_batch_size': 8,
    'focal_alpha': 0.75,
    'focal_gamma': 3.0,
    'augment_minority': False,
    'use_mixup': False,
    'use_amp': True,
    'use_cache': True,
    'use_batched_inner': True,
    'gradient_accumulation_steps': 2
}

input_dim = OPTIMIZED_CONFIG['input_dim']
hidden_dims = OPTIMIZED_CONFIG['hidden_dims']
embedding_dim = OPTIMIZED_CONFIG['embedding_dim']
dropout = OPTIMIZED_CONFIG['dropout']

In [57]:
class ImprovedMetaLearner(nn.Module):
    def __init__(self, input_dim, hidden_dims, embedding_dim, dropout, num_classes):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.embedding_dim = embedding_dim
        self.dropout = dropout
        self.num_classes = num_classes
        
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.BatchNorm1d(hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.res_blocks = nn.ModuleList([
            ResidualBlock(hidden_dims[0], dropout) for _ in range(2)
        ])
        
        self.encoder = nn.Sequential(
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.BatchNorm1d(hidden_dims[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dims[1], embedding_dim)
        )
        
        self.classifier = classifier_head(embedding_dim, num_classes, dropout)
    
    def forward(self, x):
        x = self.input_proj(x)
        for block in self.res_blocks:
            x = block(x)
        embeddings = self.encoder(x)
        return embeddings
    
    def predict(self, x):
        embeddings = self.forward(x)
        logits = self.classifier(embeddings)
        return logits

class MetaLearner(nn.Module):
    def __init__(self, input_dim, hidden_dims, embedding_dim, dropout, num_classes):
        super(MetaLearner, self).__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.embedding_dim = embedding_dim
        self.dropout = dropout
        self.num_classes = num_classes
        
        self.encoder = meta_network(input_dim, hidden_dims, embedding_dim, dropout)
        self.classifier = classifier_head(embedding_dim, num_classes, dropout)
    
    def forward(self, x):
        embeddings = self.encoder(x)
        return embeddings
    
    def predict(self, x):
        embeddings = self.encoder(x)
        logits = self.classifier(embeddings)
        return logits

model = ImprovedMetaLearner(input_dim, hidden_dims, embedding_dim, dropout, num_classes).to(device)

# Compile model for PyTorch 2.0+ speedup (disabled during training to avoid state_dict issues)
# Will compile for inference after loading checkpoint
compile_model = False
if hasattr(torch, 'compile') and compile_model:
    try:
        model = torch.compile(model, mode='reduce-overhead')
        print("✓ Model compiled with torch.compile")
    except Exception as e:
        print(f"torch.compile not available: {e}")

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Encoder parameters: {sum(p.numel() for p in model.encoder.parameters()):,}")
print(f"Classifier parameters: {sum(p.numel() for p in model.classifier.parameters()):,}")

QUICK_TEST = False

if QUICK_TEST:
    meta_lr = 5e-4
    inner_lr = 5e-3
    inner_steps = 5
    meta_batch_size = 4
    num_meta_iterations = 50
    k_shot_minority = 3
    k_shot_majority = 5
    q_query = 10
    early_stopping_patience = 20
    min_delta = 1e-4
    print("\nQUICK TEST MODE ENABLED")
else:
    meta_lr = OPTIMIZED_CONFIG['meta_lr']
    inner_lr = OPTIMIZED_CONFIG['inner_lr']
    inner_steps = OPTIMIZED_CONFIG['inner_steps']
    meta_batch_size = OPTIMIZED_CONFIG['meta_batch_size']
    num_meta_iterations = 1000
    k_shot_minority = 5
    k_shot_majority = 10
    q_query = 15
    early_stopping_patience = 100
    min_delta = 1e-4

meta_optimizer = Adam(model.parameters(), lr=meta_lr)

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
scheduler = CosineAnnealingWarmRestarts(meta_optimizer, T_0=100, T_mult=2, eta_min=1e-5)

# Initialize gradient scaler for mixed precision
# Note: GradScaler disabled for meta-training loop to avoid complexity
# Mixed precision still used in inner loop via autocast
grad_scaler = None

# Initialize episode cache
episode_cache = EpisodeCache(cache_size=500) if OPTIMIZED_CONFIG['use_cache'] else None

print(f"\nMeta-Learning Configuration:")
print(f"  Meta LR: {meta_lr}")
print(f"  Inner LR: {inner_lr}")
print(f"  Inner steps: {inner_steps}")
print(f"  Meta batch size: {meta_batch_size}")
print(f"  Iterations: {num_meta_iterations}")
print(f"  K-shot (minority): {k_shot_minority}")
print(f"  K-shot (majority): {k_shot_majority}")
print(f"  Query samples: {q_query}")
print(f"  Early stopping patience: {early_stopping_patience}")
print(f"  LR scheduler: CosineAnnealingWarmRestarts")

Model parameters: 1,331,778
Encoder parameters: 164,736
Classifier parameters: 8,386

Meta-Learning Configuration:
  Meta LR: 0.0005
  Inner LR: 0.005
  Inner steps: 10
  Meta batch size: 16
  Iterations: 1000
  K-shot (minority): 5
  K-shot (majority): 10
  Query samples: 15
  Early stopping patience: 100
  LR scheduler: CosineAnnealingWarmRestarts


In [58]:
train_sources = []
source_imbalance_ratios = {}

for source_name, source_data in data_dict.items():
    if source_data['labels'] is not None:
        labels = source_data['labels']
        if len(np.unique(labels)) >= 2:
            train_sources.append(source_name)
            unique, counts = np.unique(labels, return_counts=True)
            imb_ratio = counts.max() / counts.min() if len(counts) > 1 else 1.0
            source_imbalance_ratios[source_name] = imb_ratio

train_sources_sorted = sorted(train_sources, key=lambda x: source_imbalance_ratios[x])

meta_train_sources = train_sources_sorted[:int(len(train_sources_sorted) * 0.8)]
meta_val_sources = train_sources_sorted[int(len(train_sources_sorted) * 0.8):]

print(f"\nTraining sources: {len(train_sources)}")
print(f"Meta-train sources: {len(meta_train_sources)}")
print(f"Meta-val sources: {len(meta_val_sources)}")
print(f"Sources (sorted by imbalance): {train_sources_sorted}")
print(f"Imbalance ratios: {[(s, f'{source_imbalance_ratios[s]:.1f}:1') for s in train_sources_sorted[:5]]}")


feat_variant = 'selected_imbalanced'

source_features = {}
source_labels = {}
source_scalers = {}
source_k_shots = {}

for source_name in train_sources:
    source_data = data_dict[source_name]
    if feat_variant in source_data['feature_variants']:
        X = source_data['feature_variants'][feat_variant]
        y = source_data['labels']
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        source_features[source_name] = X_scaled
        source_labels[source_name] = y
        source_scalers[source_name] = scaler
        
        unique, counts = np.unique(y, return_counts=True)
        imb_ratio = counts.max() / counts.min() if len(counts) > 1 else 1.0
        minority_count = counts.min()
        
        k_min = max(2, min(k_shot_minority, minority_count // 4))
        k_maj = max(k_min, min(k_shot_majority, minority_count // 2))
        source_k_shots[source_name] = {'minority': k_min, 'majority': k_maj}
        
        print(f"  {source_name}: {len(y)} samples, imbalance {imb_ratio:.2f}:1, k-shot {k_min}/{k_maj}")

print(f"\nPrepared {len(source_features)} sources for meta-training")


Training sources: 13
Meta-train sources: 10
Meta-val sources: 3
Sources (sorted by imbalance): ['HPC_2k', 'Windows_2k', 'Hadoop_2k', 'Apache_2k', 'Zookeeper_2k', 'Mac_2k', 'Thunderbird_2k', 'BGL_2k', 'Proxifier_2k', 'Linux_2k', 'Android_2k', 'HealthApp_2k', 'Spark_2k']
Imbalance ratios: [('HPC_2k', '1.2:1'), ('Windows_2k', '1.5:1'), ('Hadoop_2k', '1.9:1'), ('Apache_2k', '2.5:1'), ('Zookeeper_2k', '2.9:1')]
  Android_2k: 2000 samples, imbalance 75.92:1, k-shot 5/10
  Apache_2k: 2000 samples, imbalance 2.50:1, k-shot 5/10
  BGL_2k: 2000 samples, imbalance 10.90:1, k-shot 5/10
  Hadoop_2k: 2000 samples, imbalance 1.90:1, k-shot 5/10
  HealthApp_2k: 2000 samples, imbalance 180.82:1, k-shot 2/5
  HPC_2k: 2000 samples, imbalance 1.25:1, k-shot 5/10
  Linux_2k: 2000 samples, imbalance 20.28:1, k-shot 5/10
  Mac_2k: 2000 samples, imbalance 3.62:1, k-shot 5/10
  Proxifier_2k: 2000 samples, imbalance 19.62:1, k-shot 5/10
  Spark_2k: 2000 samples, imbalance 249.00:1, k-shot 2/4
  Thunderbird_2k:

In [59]:
print("\nStarting Meta-Training with speed optimizations...")

best_meta_loss = float('inf')
best_val_loss = float('inf')
meta_losses = []
val_losses = []
patience_counter = 0
curriculum_phase = 0
curriculum_thresholds = [5, 20, 100]
adaptive_lr_scheduler = TaskAdaptiveLR(base_lr=inner_lr)
gradient_accumulation_steps = OPTIMIZED_CONFIG['gradient_accumulation_steps']

print(f"Speed optimizations enabled:")
print(f"  - Mixed Precision (AMP): {OPTIMIZED_CONFIG['use_amp'] and torch.cuda.is_available()}")
print(f"  - Episode Caching: {OPTIMIZED_CONFIG['use_cache']}")
print(f"  - Batched Inner Loop: {OPTIMIZED_CONFIG['use_batched_inner']}")
print(f"  - Gradient Accumulation: {gradient_accumulation_steps}x")
print(f"  - CUDNN Benchmark: {torch.backends.cudnn.benchmark}")

def get_curriculum_sources(phase, sources_sorted, imbalance_ratios):
    if phase == 0:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[0]]
    elif phase == 1:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[1]]
    elif phase == 2:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[2]]
    else:
        return sources_sorted

def evaluate_meta_validation(model, val_sources, source_features, source_labels, source_k_shots, q_query):
    """Evaluate on meta-validation set"""
    model.eval()
    val_task_losses = []
    
    for source_name in val_sources:
        if source_name not in source_features:
            continue
        
        X_source = source_features[source_name]
        y_source = source_labels[source_name]
        k_shots = source_k_shots[source_name]
        
        episode = create_balanced_imbalanced_episode(X_source, y_source, q_query=q_query)
        if episode[0] is None:
            continue
        
        support_X, support_y, query_X, query_y = episode
        
        # Inner loop needs gradients for adaptation
        adapted_model = maml_inner_loop(model, support_X, support_y, inner_lr, inner_steps, focal_loss)
        
        # Evaluate on query set without gradients
        adapted_model.eval()
        with torch.no_grad():
            
            query_X_tensor = torch.FloatTensor(query_X).to(device)
            query_y_tensor = torch.LongTensor(query_y).to(device)
            
            query_embeddings = adapted_model(query_X_tensor)
            query_logits = adapted_model.classifier(query_embeddings)
            prototypes = refined_prototypes(query_embeddings, query_y_tensor)
            
            task_loss = combined_meta_loss(query_embeddings, query_logits, query_y_tensor, prototypes)
            val_task_losses.append(task_loss.item())
    
    model.train()
    return np.mean(val_task_losses) if val_task_losses else float('inf')

curriculum_sources = get_curriculum_sources(curriculum_phase, meta_train_sources, source_imbalance_ratios)
if not curriculum_sources:
    curriculum_sources = meta_train_sources[:len(meta_train_sources)//2]


Starting Meta-Training with speed optimizations...
Speed optimizations enabled:
  - Mixed Precision (AMP): True
  - Episode Caching: True
  - Batched Inner Loop: False
  - Gradient Accumulation: 1x
  - CUDNN Benchmark: True


In [60]:
print(f"Starting with curriculum phase {curriculum_phase}: {len(curriculum_sources)} sources")

for iteration in range(num_meta_iterations):
    if iteration > 0 and iteration % 200 == 0 and curriculum_phase < 3:
        curriculum_phase += 1
        curriculum_sources = get_curriculum_sources(curriculum_phase, meta_train_sources, source_imbalance_ratios)
        if not curriculum_sources:
            curriculum_sources = meta_train_sources
        print(f"\nCurriculum phase {curriculum_phase}: {len(curriculum_sources)} sources")
        
        if episode_cache:
            print(f"  {episode_cache.get_stats()}")
    
    # Reset optimizer gradients
    meta_optimizer.zero_grad()
    
    # Gradient accumulation loop
    all_task_losses = []
    total_valid_tasks = 0
    
    for accum_step in range(gradient_accumulation_steps):
        task_losses = []
        
        for batch_idx in range(meta_batch_size):
            available_sources = [s for s in curriculum_sources if s in source_features]
            if not available_sources:
                available_sources = list(source_features.keys())
            
            source_name = np.random.choice(available_sources)
            X_source = source_features[source_name]
            y_source = source_labels[source_name]
            k_shots = source_k_shots[source_name]
            imb_ratio = source_imbalance_ratios[source_name]
            
            adaptive_steps = adaptive_inner_steps(imb_ratio, inner_steps)
            adaptive_lr = adaptive_lr_scheduler.get_lr(imb_ratio)
            
            # Use cached episodes if enabled
            if episode_cache:
                episode = episode_cache.get_episode(
                    source_name, X_source, y_source,
                    k_shots['minority'], k_shots['majority'], q_query
                )
            else:
                episode = create_balanced_imbalanced_episode(X_source, y_source, q_query=q_query)
            
            if episode[0] is None:
                continue
            
            support_X, support_y, query_X, query_y = episode
            
            if OPTIMIZED_CONFIG['augment_minority']:
                support_X, support_y = augment_support_set(support_X, support_y, augment_factor=2)
            
            # Use batched inner loop if enabled
            if OPTIMIZED_CONFIG['use_batched_inner']:
                adapted_model = batched_maml_inner_loop(model, support_X, support_y, 
                                                       adaptive_lr, adaptive_steps, focal_loss)
            else:
                adapted_model = maml_inner_loop(model, support_X, support_y, 
                                               adaptive_lr, adaptive_steps, focal_loss,
                                               use_amp=OPTIMIZED_CONFIG['use_amp'])
            
            query_X_tensor = torch.FloatTensor(query_X).to(device, non_blocking=True)
            query_y_tensor = torch.LongTensor(query_y).to(device, non_blocking=True)
            
            # Use mixed precision for query
            if OPTIMIZED_CONFIG['use_amp'] and torch.cuda.is_available():
                device_type = 'cuda' if USE_NEW_AMP_API else None
                with autocast(device_type=device_type) if USE_NEW_AMP_API else autocast():
                    query_embeddings = adapted_model(query_X_tensor)
                    query_logits = adapted_model.classifier(query_embeddings)
                    prototypes = refined_prototypes(query_embeddings, query_y_tensor)
                    task_loss = combined_meta_loss(query_embeddings, query_logits, query_y_tensor, prototypes)
            else:
                query_embeddings = adapted_model(query_X_tensor)
                query_logits = adapted_model.classifier(query_embeddings)
                prototypes = refined_prototypes(query_embeddings, query_y_tensor)
                task_loss = combined_meta_loss(query_embeddings, query_logits, query_y_tensor, prototypes)
            
            # Scale loss for gradient accumulation
            scaled_loss = task_loss / gradient_accumulation_steps
            task_losses.append(scaled_loss)
            total_valid_tasks += 1
        
        if task_losses:
            meta_loss = torch.stack(task_losses).mean()
            all_task_losses.append(meta_loss)
            
            # Backward pass (accumulate gradients)
            meta_loss.backward()
    
    # Update after accumulation
    if total_valid_tasks > 0 and all_task_losses:
        # Compute average loss for logging
        current_loss = torch.stack(all_task_losses).mean().item() * gradient_accumulation_steps
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Step optimizer
        meta_optimizer.step()
        scheduler.step()
        meta_losses.append(current_loss)
        
        if (iteration + 1) % 100 == 0 and meta_val_sources:
            val_loss = evaluate_meta_validation(model, meta_val_sources, source_features, 
                                               source_labels, source_k_shots, q_query)
            val_losses.append(val_loss)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                # Clean state dict before saving
                save_state_dict = model.state_dict()
                if any(key.startswith('_orig_mod.') for key in save_state_dict.keys()):
                    save_state_dict = {key.replace('_orig_mod.', ''): value for key, value in save_state_dict.items()}
                
                torch.save({
                    'model': save_state_dict,
                    'iteration': iteration,
                    'meta_loss': current_loss,
                    'val_loss': val_loss,
                    'curriculum_phase': curriculum_phase
                }, MODELS_PATH / 'best_meta_model.pt')
            else:
                patience_counter += 1
        elif current_loss < best_meta_loss - min_delta:
            best_meta_loss = current_loss
            patience_counter = 0
            # Clean state dict before saving
            save_state_dict = model.state_dict()
            if any(key.startswith('_orig_mod.') for key in save_state_dict.keys()):
                save_state_dict = {key.replace('_orig_mod.', ''): value for key, value in save_state_dict.items()}
            
            torch.save({
                'model': save_state_dict,
                'iteration': iteration,
                'meta_loss': best_meta_loss,
                'curriculum_phase': curriculum_phase
            }, MODELS_PATH / 'best_meta_model.pt')
        else:
            patience_counter += 1
        
        if (iteration + 1) % 50 == 0:
            avg_loss = np.mean(meta_losses[-50:])
            current_lr = meta_optimizer.param_groups[0]['lr']
            val_info = f", Val: {val_losses[-1]:.4f}" if val_losses else ""
            cache_info = f", {episode_cache.get_stats()}" if episode_cache and (iteration + 1) % 200 == 0 else ""
            print(f"Iter {iteration + 1}/{num_meta_iterations} - Loss: {avg_loss:.4f}{val_info}, LR: {current_lr:.6f}, Patience: {patience_counter}/{early_stopping_patience}{cache_info}")
        
        if patience_counter >= early_stopping_patience:
            print(f"\nEarly stopping at iteration {iteration + 1}")
            break
    
    # Clear cache periodically
    if torch.cuda.is_available() and (iteration + 1) % 100 == 0:
        torch.cuda.empty_cache()

print(f"\nMeta-training complete. Best meta loss: {best_meta_loss:.4f}")
if val_losses:
    print(f"Best validation loss: {best_val_loss:.4f}")


checkpoint = torch.load(MODELS_PATH / 'best_meta_model.pt')

# Handle torch.compile state dict (removes _orig_mod. prefix)
state_dict = checkpoint['model']
if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
    state_dict = {key.replace('_orig_mod.', ''): value for key, value in state_dict.items()}

model.load_state_dict(state_dict)
print(f"Loaded best meta-model from iteration {checkpoint['iteration']}")

Starting with curriculum phase 0: 6 sources
Iter 50/1000 - Loss: 1.1919, LR: 0.000255, Patience: 7/100
Iter 100/1000 - Loss: 1.1940, Val: 0.9740, LR: 0.000500, Patience: 0/100
Iter 150/1000 - Loss: 1.1941, Val: 0.9740, LR: 0.000428, Patience: 37/100
Iter 200/1000 - Loss: 1.1941, Val: 0.9836, LR: 0.000255, Patience: 87/100, Cache hit rate: 15.22% (487/3200)

Curriculum phase 1: 9 sources
  Cache hit rate: 15.22% (487/3200)

Early stopping at iteration 213

Meta-training complete. Best meta loss: 1.1812
Best validation loss: 0.9740
Loaded best meta-model from iteration 112


In [61]:
print("\nEvaluating on test sources...")

test_results = []

test_splits = splits[:3] if QUICK_TEST else splits

for split in test_splits:
    test_source = split['test_source']
    train_sources_split = split['train_sources']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None:
        continue
    
    if feat_variant not in test_data['feature_variants']:
        continue
    
    X_test = test_data['feature_variants'][feat_variant]
    y_test = test_data['labels']
    
    if len(np.unique(y_test)) < 2:
        continue
    
    scaler_test = StandardScaler()
    X_test_scaled = scaler_test.fit_transform(X_test)
    
    X_train_list = []
    y_train_list = []
    
    for src in train_sources_split:
        if src in source_features:
            X_train_list.append(source_features[src])
            y_train_list.append(source_labels[src])
    
    if not X_train_list:
        continue
    
    X_train_combined = np.vstack(X_train_list)
    y_train_combined = np.concatenate(y_train_list)
    
    unique, counts = np.unique(y_train_combined, return_counts=True)
    minority_count = counts.min()
    
    k_shot_adapt = min(k_shot_minority, minority_count // 2)
    if k_shot_adapt < 2:
        continue
    
    episode = create_imbalanced_episode(
        X_train_combined, y_train_combined,
        k_shot_adapt, k_shot_adapt * 2, 10
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    imb_ratio = source_imbalance_ratios.get(test_source, 10.0)
    adaptive_steps = adaptive_inner_steps(imb_ratio, inner_steps)
    adaptive_lr = adaptive_lr_scheduler.get_lr(imb_ratio)
    
    if OPTIMIZED_CONFIG['augment_minority']:
        support_X, support_y = augment_support_set(support_X, support_y, augment_factor=2)
    
    adapted_model = maml_inner_loop(
        model, support_X, support_y,
        adaptive_lr, adaptive_steps, focal_loss
    )
    
    adapted_model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        test_logits = adapted_model.predict(X_test_tensor)
        test_probs = F.softmax(test_logits, dim=1).cpu().numpy()
        test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test, test_preds, test_probs)
    
    test_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'test_samples': len(y_test),
        'support_samples': len(support_y)
    })
    
    print(f"\n{test_source}:")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")
    print(f"  MCC: {metrics['mcc']:.4f}")


if test_results:
    df_results = pd.DataFrame(test_results)
    df_results = df_results.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("META-LEARNING EVALUATION SUMMARY")
    print("="*80)
    print(df_results.to_string(index=False))
    
    print("\n" + "="*60)
    print("AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(test_results)}")
    print(f"Average F1-Macro: {df_results['f1_macro'].mean():.4f} ± {df_results['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_results['balanced_acc'].mean():.4f} ± {df_results['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_results['auroc'].mean():.4f} ± {df_results['auroc'].std():.4f}")
    print(f"Average MCC: {df_results['mcc'].mean():.4f} ± {df_results['mcc'].std():.4f}")
    
    results_file = RESULTS_PATH / f"meta_learning_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(results_file, index=False)
    print(f"\nResults saved to: {results_file}")
    
    with open(RESULTS_PATH / 'meta_learning_summary.pkl', 'wb') as f:
        pickle.dump({
            'results': test_results,
            'meta_losses': meta_losses,
            'config': {
                'meta_lr': meta_lr,
                'inner_lr': inner_lr,
                'inner_steps': inner_steps,
                'meta_batch_size': meta_batch_size,
                'num_meta_iterations': num_meta_iterations,
                'k_shot_minority': k_shot_minority,
                'k_shot_majority': k_shot_majority,
                'q_query': q_query,
                'input_dim': input_dim,
                'hidden_dims': hidden_dims,
                'embedding_dim': embedding_dim
            },
            'timestamp': datetime.now().isoformat()
        }, f)
    print(f"Summary saved to: {RESULTS_PATH / 'meta_learning_summary.pkl'}")

else:
    print("\nNo test results generated")


Evaluating on test sources...

Android_2k:
  F1-Macro: 0.4967
  Balanced Acc: 0.5000
  AUROC: 0.2482
  MCC: 0.0000

Apache_2k:
  F1-Macro: 0.4167
  Balanced Acc: 0.5000
  AUROC: 0.6217
  MCC: 0.0000

BGL_2k:
  F1-Macro: 0.4781
  Balanced Acc: 0.5000
  AUROC: 0.2732
  MCC: 0.0000

Hadoop_2k:
  F1-Macro: 0.2565
  Balanced Acc: 0.5000
  AUROC: 0.4747
  MCC: 0.0000

HealthApp_2k:
  F1-Macro: 0.4982
  Balanced Acc: 0.4992
  AUROC: 0.4245
  MCC: -0.0029

HPC_2k:
  F1-Macro: 0.3569
  Balanced Acc: 0.5000
  AUROC: 0.5452
  MCC: 0.0000

Linux_2k:
  F1-Macro: 0.0578
  Balanced Acc: 0.5063
  AUROC: 0.8692
  MCC: 0.0245

Mac_2k:
  F1-Macro: 0.4393
  Balanced Acc: 0.5000
  AUROC: 0.4877
  MCC: 0.0000

Proxifier_2k:
  F1-Macro: 0.5080
  Balanced Acc: 0.5103
  AUROC: 0.9972
  MCC: 0.1401

Spark_2k:
  F1-Macro: 0.4987
  Balanced Acc: 0.4995
  AUROC: 0.5059
  MCC: -0.0020

Thunderbird_2k:
  F1-Macro: 0.4744
  Balanced Acc: 0.5000
  AUROC: 0.1980
  MCC: 0.0000

Windows_2k:
  F1-Macro: 0.3768
  Balanced

In [62]:
print("\nPrototypical Network Evaluation...")

prototypical_results = []

model.eval()

proto_splits = splits[:3] if QUICK_TEST else splits

for split in proto_splits:
    test_source = split['test_source']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None or feat_variant not in test_data['feature_variants']:
        continue
    
    X_test = test_data['feature_variants'][feat_variant]
    y_test = test_data['labels']
    
    if len(np.unique(y_test)) < 2:
        continue
    
    scaler_test = StandardScaler()
    X_test_scaled = scaler_test.fit_transform(X_test)
    
    X_train_list = []
    y_train_list = []
    
    for src in split['train_sources']:
        if src in source_features:
            X_train_list.append(source_features[src])
            y_train_list.append(source_labels[src])
    
    if not X_train_list:
        continue
    
    X_train_combined = np.vstack(X_train_list)
    y_train_combined = np.concatenate(y_train_list)
    
    unique, counts = np.unique(y_train_combined, return_counts=True)
    minority_count = counts.min()
    k_proto = min(20, minority_count // 2)
    
    if k_proto < 5:
        continue
    
    episode = create_imbalanced_episode(
        X_train_combined, y_train_combined,
        k_proto, k_proto * 2, 0
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    with torch.no_grad():
        support_X_tensor = torch.FloatTensor(support_X).to(device)
        support_y_tensor = torch.LongTensor(support_y).to(device)
        support_embeddings = model(support_X_tensor)
        
        prototypes, proto_classes = compute_prototypes(support_embeddings, support_y_tensor)
        
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        test_embeddings = model(X_test_tensor)
        
        distances = torch.cdist(test_embeddings, prototypes, p=2)
        test_preds = torch.argmin(distances, dim=1).cpu().numpy()
        
        probs = F.softmax(-distances, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test, test_preds, probs)
    
    prototypical_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'test_samples': len(y_test),
        'prototypes': len(prototypes)
    })
    
    print(f"\n{test_source} (Prototypical):")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")


if prototypical_results:
    df_proto = pd.DataFrame(prototypical_results)
    df_proto = df_proto.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("PROTOTYPICAL NETWORK EVALUATION SUMMARY")
    print("="*80)
    print(df_proto.to_string(index=False))
    
    print("\n" + "="*60)
    print("PROTOTYPICAL AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(prototypical_results)}")
    print(f"Average F1-Macro: {df_proto['f1_macro'].mean():.4f} ± {df_proto['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_proto['balanced_acc'].mean():.4f} ± {df_proto['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_proto['auroc'].mean():.4f} ± {df_proto['auroc'].std():.4f}")
    
    proto_file = RESULTS_PATH / f"prototypical_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_proto.to_csv(proto_file, index=False)
    print(f"\nPrototypical results saved to: {proto_file}")


Prototypical Network Evaluation...

Android_2k (Prototypical):
  F1-Macro: 0.4056
  Balanced Acc: 0.6046
  AUROC: 0.6564

Apache_2k (Prototypical):
  F1-Macro: 0.9793
  Balanced Acc: 0.9834
  AUROC: 0.9787

BGL_2k (Prototypical):
  F1-Macro: 0.3397
  Balanced Acc: 0.2808
  AUROC: 0.1349

Hadoop_2k (Prototypical):
  F1-Macro: 0.4480
  Balanced Acc: 0.5421
  AUROC: 0.2930

HealthApp_2k (Prototypical):
  F1-Macro: 0.4832
  Balanced Acc: 0.6629
  AUROC: 0.7672

HPC_2k (Prototypical):
  F1-Macro: 0.2215
  Balanced Acc: 0.2552
  AUROC: 0.1528

Linux_2k (Prototypical):
  F1-Macro: 0.2813
  Balanced Acc: 0.5841
  AUROC: 0.7102

Mac_2k (Prototypical):
  F1-Macro: 0.4730
  Balanced Acc: 0.5350
  AUROC: 0.5774

Proxifier_2k (Prototypical):
  F1-Macro: 0.5653
  Balanced Acc: 0.8361
  AUROC: 0.7984

Spark_2k (Prototypical):
  F1-Macro: 0.4116
  Balanced Acc: 0.8331
  AUROC: 0.9584

Thunderbird_2k (Prototypical):
  F1-Macro: 0.6657
  Balanced Acc: 0.7987
  AUROC: 0.8642

Windows_2k (Prototypical):


In [63]:
print("\nFew-Shot Transfer Learning Evaluation...")

transfer_results = []

transfer_splits = splits[:3] if QUICK_TEST else splits

for split in transfer_splits:
    test_source = split['test_source']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None or feat_variant not in test_data['feature_variants']:
        continue
    
    X_test_full = test_data['feature_variants'][feat_variant]
    y_test_full = test_data['labels']
    
    if len(np.unique(y_test_full)) < 2:
        continue
    
    scaler_transfer = StandardScaler()
    X_test_scaled = scaler_transfer.fit_transform(X_test_full)
    
    unique, counts = np.unique(y_test_full, return_counts=True)
    minority_count = counts.min()
    k_transfer = min(10, minority_count // 3)
    
    if k_transfer < 3:
        continue
    
    X_train_transfer, X_test_transfer, y_train_transfer, y_test_transfer = train_test_split(
        X_test_scaled, y_test_full, test_size=0.7, random_state=SEED, stratify=y_test_full
    )
    
    episode = create_imbalanced_episode(
        X_train_transfer, y_train_transfer,
        k_transfer, k_transfer * 2, 5
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    transfer_model = ImprovedMetaLearner(input_dim, hidden_dims, embedding_dim, dropout, num_classes).to(device)
    
    # Handle torch.compile state dict
    state_dict = model.state_dict()
    if any(key.startswith('_orig_mod.') for key in state_dict.keys()):
        state_dict = {key.replace('_orig_mod.', ''): value for key, value in state_dict.items()}
    
    transfer_model.load_state_dict(state_dict)
    
    transfer_optimizer = Adam(transfer_model.parameters(), lr=1e-3)
    
    support_X_tensor = torch.FloatTensor(support_X).to(device)
    support_y_tensor = torch.LongTensor(support_y).to(device)
    
    transfer_model.train()
    for epoch in range(50):
        transfer_optimizer.zero_grad()
        logits = transfer_model.predict(support_X_tensor)
        loss = focal_loss(logits, support_y_tensor)
        loss.backward()
        transfer_optimizer.step()
    
    transfer_model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_transfer).to(device)
        test_logits = transfer_model.predict(X_test_tensor)
        test_probs = F.softmax(test_logits, dim=1).cpu().numpy()
        test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test_transfer, test_preds, test_probs)
    
    transfer_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'train_samples': len(support_y),
        'test_samples': len(y_test_transfer)
    })
    
    print(f"\n{test_source} (Transfer):")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")
    
    del transfer_model
    torch.cuda.empty_cache()


if transfer_results:
    df_transfer = pd.DataFrame(transfer_results)
    df_transfer = df_transfer.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("TRANSFER LEARNING EVALUATION SUMMARY")
    print("="*80)
    print(df_transfer.to_string(index=False))
    
    print("\n" + "="*60)
    print("TRANSFER AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(transfer_results)}")
    print(f"Average F1-Macro: {df_transfer['f1_macro'].mean():.4f} ± {df_transfer['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_transfer['balanced_acc'].mean():.4f} ± {df_transfer['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_transfer['auroc'].mean():.4f} ± {df_transfer['auroc'].std():.4f}")
    
    transfer_file = RESULTS_PATH / f"transfer_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_transfer.to_csv(transfer_file, index=False)
    print(f"\nTransfer results saved to: {transfer_file}")


Few-Shot Transfer Learning Evaluation...

Apache_2k (Transfer):
  F1-Macro: 0.9913
  Balanced Acc: 0.9950
  AUROC: 1.0000

BGL_2k (Transfer):
  F1-Macro: 0.9819
  Balanced Acc: 0.9930
  AUROC: 0.9991

Hadoop_2k (Transfer):
  F1-Macro: 0.9627
  Balanced Acc: 0.9602
  AUROC: 0.9965

HPC_2k (Transfer):
  F1-Macro: 0.9870
  Balanced Acc: 0.9876
  AUROC: 0.9994

Linux_2k (Transfer):
  F1-Macro: 0.9494
  Balanced Acc: 0.9948
  AUROC: 0.9998

Mac_2k (Transfer):
  F1-Macro: 0.9283
  Balanced Acc: 0.9217
  AUROC: 0.9784

Proxifier_2k (Transfer):
  F1-Macro: 1.0000
  Balanced Acc: 1.0000
  AUROC: 1.0000

Thunderbird_2k (Transfer):
  F1-Macro: 0.9822
  Balanced Acc: 0.9964
  AUROC: 0.9999

Windows_2k (Transfer):
  F1-Macro: 0.9859
  Balanced Acc: 0.9872
  AUROC: 0.9997

Zookeeper_2k (Transfer):
  F1-Macro: 0.9376
  Balanced Acc: 0.9202
  AUROC: 0.9825

TRANSFER LEARNING EVALUATION SUMMARY
   test_source  f1_macro  balanced_acc    auroc      mcc  train_samples  test_samples
  Proxifier_2k  1.0000

In [64]:
if test_results and prototypical_results and transfer_results:
    comparison_data = []
    
    sources_all = set([r['test_source'] for r in test_results])
    sources_proto = set([r['test_source'] for r in prototypical_results])
    sources_transfer = set([r['test_source'] for r in transfer_results])
    common_sources = sources_all & sources_proto & sources_transfer
    
    for source in common_sources:
        maml_result = next((r for r in test_results if r['test_source'] == source), None)
        proto_result = next((r for r in prototypical_results if r['test_source'] == source), None)
        transfer_result = next((r for r in transfer_results if r['test_source'] == source), None)
        
        if maml_result and proto_result and transfer_result:
            comparison_data.append({
                'Source': source,
                'MAML F1': maml_result['f1_macro'],
                'Proto F1': proto_result['f1_macro'],
                'Transfer F1': transfer_result['f1_macro'],
                'Best Method': max([
                    ('MAML', maml_result['f1_macro']),
                    ('Proto', proto_result['f1_macro']),
                    ('Transfer', transfer_result['f1_macro'])
                ], key=lambda x: x[1])[0]
            })
    
    if comparison_data:
        df_comparison = pd.DataFrame(comparison_data)
        print("\n" + df_comparison.to_string(index=False))
        
        print("\n" + "="*60)
        print("METHOD COMPARISON")
        print("="*60)
        print(f"MAML Average F1: {df_comparison['MAML F1'].mean():.4f}")
        print(f"Prototypical Average F1: {df_comparison['Proto F1'].mean():.4f}")
        print(f"Transfer Average F1: {df_comparison['Transfer F1'].mean():.4f}")
        
        best_counts = df_comparison['Best Method'].value_counts()
        print(f"\nBest method frequency:")
        for method, count in best_counts.items():
            print(f"  {method}: {count} times ({count/len(df_comparison)*100:.1f}%)")
        
        comparison_file = RESULTS_PATH / f"method_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_comparison.to_csv(comparison_file, index=False)
        print(f"\nComparison saved to: {comparison_file}")


        Source  MAML F1  Proto F1  Transfer F1 Best Method
  Proxifier_2k 0.508026  0.565335     1.000000    Transfer
    Windows_2k 0.376753  0.378944     0.985850    Transfer
  Zookeeper_2k 0.426277  0.773073     0.937597    Transfer
        HPC_2k 0.356913  0.221531     0.987001    Transfer
     Apache_2k 0.416740  0.979338     0.991315    Transfer
        BGL_2k 0.478079  0.339716     0.981907    Transfer
        Mac_2k 0.439305  0.473000     0.928329    Transfer
     Hadoop_2k 0.256506  0.447951     0.962656    Transfer
Thunderbird_2k 0.474376  0.665700     0.982199    Transfer
      Linux_2k 0.057846  0.281255     0.949417    Transfer

METHOD COMPARISON
MAML Average F1: 0.3791
Prototypical Average F1: 0.5126
Transfer Average F1: 0.9706

Best method frequency:
  Transfer: 10 times (100.0%)

Comparison saved to: C:\Computer Science\AIMLDL\log-anomaly-detection\results\meta_learning\method_comparison_20251125_031040.csv


In [65]:
final_state_dict = model.state_dict()
if any(key.startswith('_orig_mod.') for key in final_state_dict.keys()):
    final_state_dict = {key.replace('_orig_mod.', ''): value for key, value in final_state_dict.items()}

torch.save({
    'model': final_state_dict,
    'config': {
        'input_dim': input_dim,
        'hidden_dims': hidden_dims,
        'embedding_dim': embedding_dim,
        'dropout': dropout,
        'num_classes': num_classes
    },
    'meta_config': {
        'meta_lr': meta_lr,
        'inner_lr': inner_lr,
        'inner_steps': inner_steps,
        'meta_batch_size': meta_batch_size,
        'num_meta_iterations': num_meta_iterations,
        'early_stopping_patience': early_stopping_patience,
        'curriculum_learning': True
    },
    'training_info': {
        'final_iteration': len(meta_losses),
        'best_meta_loss': best_meta_loss,
        'curriculum_phases': curriculum_phase + 1
    },
    'timestamp': datetime.now().isoformat()
}, MODELS_PATH / 'final_meta_model.pt')

print(f"\nFinal model saved to: {MODELS_PATH / 'final_meta_model.pt'}")


Final model saved to: C:\Computer Science\AIMLDL\log-anomaly-detection\models\meta_learning\final_meta_model.pt


In [66]:
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].plot(meta_losses)
    axes[0, 0].set_title('Meta-Training Loss')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    if test_results:
        sources = [r['test_source'] for r in test_results]
        f1_scores = [r['f1_macro'] for r in test_results]
        axes[0, 1].barh(sources, f1_scores)
        axes[0, 1].set_title('F1-Macro by Source (MAML)')
        axes[0, 1].set_xlabel('F1-Macro')
        axes[0, 1].grid(True, alpha=0.3)
    
    if comparison_data:
        methods = ['MAML', 'Proto', 'Transfer']
        avg_f1s = [
            df_comparison['MAML F1'].mean(),
            df_comparison['Proto F1'].mean(),
            df_comparison['Transfer F1'].mean()
        ]
        axes[1, 0].bar(methods, avg_f1s)
        axes[1, 0].set_title('Average F1-Macro by Method')
        axes[1, 0].set_ylabel('F1-Macro')
        axes[1, 0].grid(True, alpha=0.3)
    
    window = 50
    if len(meta_losses) > window:
        smoothed = np.convolve(meta_losses, np.ones(window)/window, mode='valid')
        axes[1, 1].plot(smoothed)
        axes[1, 1].set_title(f'Smoothed Meta-Training Loss (window={window})')
        axes[1, 1].set_xlabel('Iteration')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(RESULTS_PATH / 'meta_learning_visualization.png', dpi=150, bbox_inches='tight')
    print(f"Visualization saved to: {RESULTS_PATH / 'meta_learning_visualization.png'}")
    plt.close()
except Exception as e:
    print(f"Visualization skipped: {e}")

Visualization saved to: C:\Computer Science\AIMLDL\log-anomaly-detection\results\meta_learning\meta_learning_visualization.png
